In [49]:
import numpy as np

In [50]:
#funkcja generująca losową macierz o rozmiarze nxn i wartościach z przedziału (0.00000001, 1.0) 

def generate_matrix(n):
    return np.random.uniform(0.00000001, 1.0,(n,n))

# Funkcja pomocnicza do znajdowania najbliższej potęgi dwójki
def next_power_of_two(n):
    return 1 if n == 0 else 2**(n - 1).bit_length()

# Funkcja rozszerzająca macierz do rozmiaru będącego potęgą dwójki
def pad_matrix_to_power_of_two(A):
    (n, m) = A.shape
    new_size = next_power_of_two(max(n, m))
    padded_matrix = np.zeros((new_size, new_size))
    padded_matrix[:n, :m] = A
    return padded_matrix

# Funkcja przycinająca macierz do oryginalnego rozmiaru
def unpad_matrix(C, original_size):
    return C[:original_size, :original_size]

In [51]:
# 1. Rekurencyjne mnożenie macierzy metodą Binéta (rekurencyjne, ale bez optymalizacji Strassena)
def binet_multiply(A, B):
    n = A.shape[0]
    if n == 1:
        return A * B
    else:
        mid = n // 2
        A11, A12, A21, A22 = A[:mid, :mid], A[:mid, mid:], A[mid:, :mid], A[mid:, mid:]
        B11, B12, B21, B22 = B[:mid, :mid], B[:mid, mid:], B[mid:, :mid], B[mid:, mid:]

        C11 = binet_multiply(A11, B11) + binet_multiply(A12, B21)
        C12 = binet_multiply(A11, B12) + binet_multiply(A12, B22)
        C21 = binet_multiply(A21, B11) + binet_multiply(A22, B21)
        C22 = binet_multiply(A21, B12) + binet_multiply(A22, B22)

        C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))
        return C

In [52]:
# 2. Rekurencyjne mnożenie macierzy metodą Strassena
def strassen_multiply(A, B):
    n = A.shape[0]
    if n == 1:
        return A * B
    else:
        mid = n // 2
        A11, A12, A21, A22 = A[:mid, :mid], A[:mid, mid:], A[mid:, :mid], A[mid:, mid:]
        B11, B12, B21, B22 = B[:mid, :mid], B[:mid, mid:], B[mid:, :mid], B[mid:, mid:]

        M0 = strassen_multiply(A11 + A22, B11 + B22)
        M1 = strassen_multiply(A21 + A22, B11)
        M2 = strassen_multiply(A11, B12 - B22)
        M3 = strassen_multiply(A22, B21 - B11)
        M4 = strassen_multiply(A11 + A12, B22)
        M5 = strassen_multiply(A21 - A11, B11 + B12)
        M6 = strassen_multiply(A12 - A22, B21 + B22)

        C11 = M0 + M3 - M4 + M6
        C12 = M2 + M4
        C21 = M1 + M3
        C22 = M0 + M2 - M1 + M5

        C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))
        return C

In [53]:
n = 10 
A = generate_matrix(n)
B = generate_matrix(n)

# Rozszerzenie macierzy do rozmiaru będącego potęgą 2
A_padded = pad_matrix_to_power_of_two(A)
B_padded = pad_matrix_to_power_of_two(B)

# Mnożenie metodą Binéta
C_binet_padded = binet_multiply(A_padded, B_padded)
C_binet = unpad_matrix(C_binet_padded, n)  # Przycinanie do oryginalnego rozmiaru
print("Mnożenie macierzy metodą Binéta (rekurencyjnie):")
print(C_binet)

# Mnożenie metodą Strassena
C_strassen_padded = strassen_multiply(A_padded, B_padded)
C_strassen = unpad_matrix(C_strassen_padded, n)  # Przycinanie do oryginalnego rozmiaru
print("\nMnożenie macierzy metodą Strassena:")
print(C_strassen)

Mnożenie macierzy metodą Binéta (rekurencyjnie):
[[3.16172529 3.59094281 2.77393093 3.88191744 4.20783079 3.12666574
  3.78640898 3.55077489 2.38472061 2.76915606]
 [2.84601472 3.46313744 2.62001417 3.67033908 3.74032784 2.71147334
  3.45770955 2.91786187 2.20274974 2.64966106]
 [2.42974928 3.28593474 2.24251405 2.71720842 3.6301185  2.76567261
  2.47225049 2.48602172 1.82788489 2.53459528]
 [2.11240023 2.70927042 2.53583579 3.09711432 3.42441963 2.14735694
  2.90195823 2.9642436  1.94397001 2.21442815]
 [2.40483404 2.26950361 2.0280613  2.50514806 2.74606361 2.06484856
  2.77003641 2.40079584 1.28420737 1.90778151]
 [2.31643393 3.37210045 2.24612237 2.6990432  3.63319338 2.52273396
  2.66744878 2.43832933 1.68312214 2.32261786]
 [1.98422634 2.79238761 2.59579348 2.56596661 3.37476357 2.15228296
  2.47851505 2.55416489 1.54960713 1.90366056]
 [1.44520284 2.19112025 2.09340825 2.50035827 2.44164377 1.72254107
  1.83288649 2.02714747 1.20783465 1.35834301]
 [1.31668853 2.03649634 1.17926